# Imports

In [1]:
from fastai.vision.all import *
from fastai.callback.fp16 import *

import pandas as pd
import numpy as np

import wandb
from fastai.callback.wandb import *

wandb.init(project="cassava", entity="teo03", config="config.yaml")

wandb: Currently logged in as: teo03 (use `wandb login --relogin` to force relogin)


# Setup

In [2]:
path_str = './data'
PATH = Path(path_str)

images_path = Path(path_str + '/train_images')
csv_path = Path(path_str + '/train.csv')

train_df = pd.read_csv(csv_path)

# Create a smaller dataset

In [3]:
msk = np.random.rand(len(train_df)) < 0.2

In [4]:
train_df = train_df[msk]
train_df['label'].value_counts(), len(train_df)

(3    2661
 2     487
 4     478
 1     412
 0     226
 Name: label, dtype: int64,
 4264)

# Data Preprocessing

In [5]:
def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

In [6]:
blocks = (ImageBlock, CategoryBlock)
splitter = RandomSplitter(valid_pct=0.2)
item_tfms = [Resize(wandb.config.image_size)]
batch_tfms = [RandomResizedCropGPU(wandb.config.image_size_gpu), *aug_transforms(flip_vert=True,
                                                         do_flip=True), Normalize.from_stats(*imagenet_stats)]

block = DataBlock(blocks=blocks, get_x=get_x, get_y=get_y, splitter=splitter, item_tfms=item_tfms, batch_tfms=batch_tfms)

In [7]:
dls = block.dataloaders(train_df, bs=wandb.config.batch_size)

# Training

In [9]:
learn = cnn_learner(dls,
                    eval(wandb.config.model),
                    loss_func=LabelSmoothingCrossEntropy(),
                    metrics=accuracy, opt_func=eval(wandb.config.optimizer),
                    cbs=[WandbCallback(), SaveModelCallback()]
                   ).to_fp16()

In [11]:
learn.fine_tune(wandb.config.epochs, base_lr=wandb.config.learning_rate, freeze_epochs=wandb.config.freeze_epochs)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: CUDA out of memory. Tried to allocate 2.13 GiB (GPU 0; 3.94 GiB total capacity; 245.61 MiB already allocated; 2.14 GiB free; 810.00 MiB reserved in total by PyTorch)